In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# loading the data from the csv file to apandas dataframe
movies_data = pd.read_csv('movies_top.csv')

In [3]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies_data.shape

(4803, 24)

In [5]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [6]:
movies = movies_data[["id", "title", "genres", "overview", "tagline","cast","director"]]
movies.head()

,id,title,genres,overview,tagline,cast,director
0,19995,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,206647,Spectre,Action Adventure Crime,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,49026,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,49529,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [7]:
movies_data.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [8]:
selected_features = ['overview','genres','keywords','tagline','cast','director']
print(selected_features)

['overview', 'genres', 'keywords', 'tagline', 'cast', 'director']


In [9]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [10]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [11]:
combined_features = movies_data['overview'] + ' ' + movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']


In [12]:
combined_features[1]

"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems. Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life At the end of the world, the adventure begins. Johnny Depp Orlando Bloom Keira Knightley Stellan Skarsg\\u00e5rd Chow Yun-fat Gore Verbinski"

In [13]:
import re
def convert_text_to_string(text):
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('\n', '', text)
    parts = text.split("['")
    if len(parts) < 2:
        return ''  # Handle the case when the splitting operation doesn't result in enough elements
   


In [14]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
combined_features = combined_features.fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(combined_features)

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(4803, 30308)

In [15]:
print(tfidf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 225554 stored elements and shape (4803, 30308)>
  Coords	Values
  (0, 239)	0.2014023731743307
  (0, 4714)	0.12769944629731542
  (0, 19924)	0.1967352296731846
  (0, 16887)	0.146320663915433
  (0, 7767)	0.18365890160698306
  (0, 18109)	0.15086902103897892
  (0, 19859)	0.38557843213681553
  (0, 28350)	0.15823322710681614
  (0, 17885)	0.11174263584921844
  (0, 27275)	0.14882005431501188
  (0, 10280)	0.14145584824717464
  (0, 19479)	0.16466647900154968
  (0, 21339)	0.1700756576010749
  (0, 1007)	0.12549607244507044
  (0, 5191)	0.16861249260632657
  (0, 558)	0.061000246221366045
  (0, 700)	0.06993405576760807
  (0, 9674)	0.08696476430201719
  (0, 23789)	0.08112409485122007
  (0, 9942)	0.08131128704508332
  (0, 6543)	0.15309637467605794
  (0, 5225)	0.15823322710681614
  (0, 10698)	0.11333311118031658
  (0, 25204)	0.2489233043475925
  (0, 29216)	0.08822543354513171
  :	:
  (4802, 9508)	0.10801504085274487
  (4802, 21202)	0.093081552

In [16]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix)

In [17]:
print(cosine_sim)

[[1.         0.02690871 0.02006298 ... 0.00422183 0.01891198 0.        ]
 [0.02690871 1.         0.01284585 ... 0.01876714 0.00946375 0.        ]
 [0.02006298 0.01284585 1.         ... 0.00887991 0.03856695 0.        ]
 ...
 [0.00422183 0.01876714 0.00887991 ... 1.         0.01336575 0.0223473 ]
 [0.01891198 0.00946375 0.03856695 ... 0.01336575 1.         0.00893746]
 [0.         0.         0.         ... 0.0223473  0.00893746 1.        ]]


In [18]:
print(cosine_sim.shape)

(4803, 4803)


In [19]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [23]:
# finding the close match for the movie name given by the user

movie_name = input('Enter your favourite movie name : ')

In [24]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Avatar']


In [25]:
close_match = find_close_match[0]
print(close_match)

Avatar


In [26]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

0


In [27]:
similarity_score = list(enumerate(cosine_sim[index_of_the_movie]))
print(similarity_score)

[(0, 1.0), (1, 0.026908712811346804), (2, 0.02006297587068585), (3, 0.017124557057359293), (4, 0.060042501791589376), (5, 0.04525038337476051), (6, 0.0057040257744829705), (7, 0.044630972722173715), (8, 0.012703339544998358), (9, 0.02346402180648177), (10, 0.038157475764169176), (11, 0.022365690251328518), (12, 0.01694301856575675), (13, 0.010999587531536884), (14, 0.042480581063179465), (15, 0.00890674528079316), (16, 0.024497545718934876), (17, 0.017499496393608205), (18, 0.04260166449401302), (19, 0.009902613332345815), (20, 0.010768346602311452), (21, 0.007406005653741268), (22, 0.01108920332974643), (23, 0.026909310066074194), (24, 0.0131199042840688), (25, 0.013694866250482636), (26, 0.05650294117224263), (27, 0.06475687611925383), (28, 0.02355647342394565), (29, 0.03390992954404001), (30, 0.03159485497885936), (31, 0.05964412710100249), (32, 0.017071645857456883), (33, 0.02243564942373817), (34, 0.0), (35, 0.04391640190432133), (36, 0.06635057495621156), (37, 0.04465880526462862

In [28]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(0, 1.0), (1531, 0.15283744208159394), (1914, 0.1499951767309461), (56, 0.1460592424042473), (94, 0.14231148047761674), (2403, 0.14189241065915828), (3158, 0.14148384884739348), (1352, 0.1396697894312811), (2198, 0.12600459028609323), (4401, 0.12085859753743189), (239, 0.1191929709163772), (47, 0.11781615719216183), (3604, 0.11739072554448464), (461, 0.11538409232272151), (661, 0.11518811524737053), (3730, 0.11206419295025222), (305, 0.11117858519581131), (4593, 0.1108974489883106), (369, 0.1103487247027977), (1354, 0.11015393709583593), (1275, 0.10586495658799477), (2229, 0.10333662569774452), (43, 0.10330381521748358), (942, 0.10260639255967492), (150, 0.10197824357974304), (432, 0.10087499043217613), (3105, 0.10081406378844683), (838, 0.10029786329022247), (1245, 0.09989194911062867), (775, 0.09955744261599626), (342, 0.09942890151289727), (4332, 0.09815011866309778), (1759, 0.097818899762023), (1053, 0.09738741368055621), (1951, 0.09488730442700159), (539, 0.09472316424576757), (7

In [29]:
len(sorted_similar_movies)

4803

In [31]:
print(combined_features[0])

In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy Science Fiction culture clash future space war space colony society Enter the World of Pandora. Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron


In [34]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(cosine_sim[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . The Grand Budapest Hotel
2 . Woman in Gold
3 . Hotel Transylvania 2
4 . Maid in Manhattan
5 . 1408
6 . Mortdecai
7 . The Best Exotic Marigold Hotel
8 . Dirty Pretty Things
9 . Girl with a Pearl Earring
10 . The Blue Room
11 . Four Rooms
12 . Bobby
13 . The Second Best Exotic Marigold Hotel
14 . Hotel for Dogs
15 . The Wendell Baker Story
16 . Before Midnight
17 . The Life Aquatic with Steve Zissou
18 . The Shining
19 . Star Wars: Clone Wars: Volume 1
20 . Sinbad: Legend of the Seven Seas
21 . By the Sea
22 . Hotel Rwanda
23 . The Beyond
24 . Kingdom of the Spiders
25 . St. Trinian's
26 . Under the Rainbow
27 . The Thomas Crown Affair
28 . The Three Musketeers
29 . Pound of Flesh
